  ## XRDrive Sim: Data Preprocessing
<img src="hand.gif">

In [20]:
import numpy as np
import time
import pandas as pd

import cv2

In [21]:
import matplotlib.pyplot as plt
%matplotlib inline

In [22]:
datasets = ['data/xrdata1','data/xrdata2','data/xrdata3','data/xrdata4']

In [23]:
import os
import copy
# MouseMode = False
MouseMode = True

data0 = []
for dataset in datasets:
    metadata = os.path.join(dataset, 'xrdata.csv')
    xrpic_path = os.path.join(dataset, 'xrpic')
    df = pd.read_csv(metadata, names=['name','state'])
    
    if MouseMode: dfv = copy.copy(df)
    
    df['name'] = [os.path.join(xrpic_path, xrpic) for xrpic in df['name']]
    data0.append(df)

    if MouseMode:
        xrpic_pathv = xrpic_path + 'v' # xrpicv
        
        if not os.path.exists(xrpic_pathv):
            os.makedirs(xrpic_pathv)

        for pic in dfv['name']:
            # make flip image
            imgFile = os.path.join(xrpic_path, xrpic)
            img = cv2.imread(imgFile)
            img = copy.deepcopy(img)
            img = cv2.flip(img, 1) # flip horizontally

            # save flip image
            imgFilev = os.path.join(xrpic_pathv, 'v_' + xrpic)
            cv2.imwrite(imgFilev, img)

        dfv['name'] = [os.path.join(xrpic_pathv, 'v_' + xrpic) for xrpic in dfv['name']]
        dfv['state'] = [ 'v' + state[-1] for state in dfv['state']]
        data0.append(dfv)
        
        

In [24]:
# join into one
joined_data = pd.concat(data0)
joined_data.head()
joined_data.to_csv('complete_data_clean', index=False)

In [25]:
num_class = len(joined_data['state'].unique())

ges_to_num = dict({(g,i) for i, g in enumerate(joined_data['state'].unique())})
num_to_ges = dict({(i,g) for i, g in enumerate(joined_data['state'].unique())})

import json
with open('ges2num.json', 'w') as fout:
    json.dump(ges_to_num, fout)

with open('num2ges.json', 'w') as fout:
    json.dump(num_to_ges, fout)

num_class, ges_to_num

(40,
 {' l1': 5,
  ' l2': 6,
  ' l3': 7,
  ' l4': 8,
  ' l5': 9,
  ' lc': 0,
  ' le': 4,
  ' lo': 1,
  ' lq': 2,
  ' lw': 3,
  ' m1': 15,
  ' m2': 16,
  ' m3': 17,
  ' m4': 18,
  ' m5': 19,
  ' mc': 10,
  ' me': 14,
  ' mo': 11,
  ' mq': 12,
  ' mw': 13,
  ' r1': 25,
  ' r2': 26,
  ' r3': 27,
  ' r4': 28,
  ' r5': 29,
  ' rc': 20,
  ' re': 24,
  ' ro': 21,
  ' rq': 22,
  ' rw': 23,
  'v1': 35,
  'v2': 36,
  'v3': 37,
  'v4': 38,
  'v5': 39,
  'vc': 30,
  've': 34,
  'vo': 31,
  'vq': 32,
  'vw': 33})

In [26]:
joined_data = joined_data.replace({'state':ges_to_num})

In [27]:
joined_data.head()

,name,state
0,data/xrdata1/xrpic/a_232bf02f_0_lc.jpg,0
1,data/xrdata1/xrpic/a_232bf02f_1_lo.jpg,1
2,data/xrdata1/xrpic/a_232bf02f_2_lq.jpg,2
3,data/xrdata1/xrpic/a_232bf02f_3_lw.jpg,3
4,data/xrdata1/xrpic/a_232bf02f_4_le.jpg,4


In [28]:
from collections import defaultdict

processed_hand = defaultdict(list)
processed_mask = defaultdict(list)

for i, (im_path, state) in enumerate(joined_data.values):
    im_name = im_path.split('/')[-1]
    
    hand_outdir = os.path.join(im_path.split('/')[0], im_path.split('/')[1], 'hand_xrpic')
    mask_outdir = os.path.join(im_path.split('/')[0], im_path.split('/')[1], 'mask_xrpic')
    
    if not os.path.exists(hand_outdir):
        os.makedirs(hand_outdir)
    
    if not os.path.exists(mask_outdir):
        os.makedirs(mask_outdir)
        
    im = cv2.imread(im_path)

    im_ycrcb = cv2.cvtColor(im, cv2.COLOR_BGR2YCR_CB)

    skin_ycrcb_mint = np.array((0, 133, 77))
    skin_ycrcb_maxt = np.array((255, 173, 127))
    skin_ycrcb = cv2.inRange(im_ycrcb, skin_ycrcb_mint, skin_ycrcb_maxt)    
    
    _, contours, _ = cv2.findContours(skin_ycrcb, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # fimd max contour
    max_area=100
    ci=0
    for i in range(len(contours)):
        cnt=contours[i]
        area = cv2.contourArea(cnt)
        if(area>max_area):
            max_area=area
            ci=i
    cnts = contours[ci]

    (x,y,w,h) = cv2.boundingRect(cnts)

    moments = cv2.moments(cnts)

    # Central mass of first order moments
    if moments['m00']!=0:
        cx = int(moments['m10']/moments['m00']) # cx = M10/M00
        cy = int(moments['m01']/moments['m00']) # cy = M01/M00
    centerMass=(cx,cy)    

    #Draw center mass
    cv2.circle(im,centerMass,1,[100,0,255],1)
    crop = im[y:y+h, x:x+w]
#     f_im = cv2.resize(crop,(200, 200), interpolation=cv2.INTER_CUBIC)
    f_im = cv2.resize(crop,(64, 64), interpolation=cv2.INTER_CUBIC)
    
    hand_out_path = os.path.join(hand_outdir, im_name)
    mask_out_path = os.path.join(mask_outdir, im_name)
    
    cv2.imwrite(hand_out_path, f_im)
    cv2.imwrite(mask_out_path, skin_ycrcb)
    
    processed_hand['name'].append(hand_out_path)
    processed_hand['state'].append(state)
    processed_mask['name'].append(mask_out_path)
    processed_mask['state'].append(state)
    
    print hand_out_path, mask_out_path

data/xrdata1/hand_xrpic/a_232bf02f_0_lc.jpg data/xrdata1/mask_xrpic/a_232bf02f_0_lc.jpg
data/xrdata1/hand_xrpic/a_232bf02f_1_lo.jpg data/xrdata1/mask_xrpic/a_232bf02f_1_lo.jpg
data/xrdata1/hand_xrpic/a_232bf02f_2_lq.jpg data/xrdata1/mask_xrpic/a_232bf02f_2_lq.jpg
data/xrdata1/hand_xrpic/a_232bf02f_3_lw.jpg data/xrdata1/mask_xrpic/a_232bf02f_3_lw.jpg
data/xrdata1/hand_xrpic/a_232bf02f_4_le.jpg data/xrdata1/mask_xrpic/a_232bf02f_4_le.jpg
data/xrdata1/hand_xrpic/a_232bf02f_5_l1.jpg data/xrdata1/mask_xrpic/a_232bf02f_5_l1.jpg
data/xrdata1/hand_xrpic/a_232bf02f_6_l2.jpg data/xrdata1/mask_xrpic/a_232bf02f_6_l2.jpg
data/xrdata1/hand_xrpic/a_232bf02f_7_l3.jpg data/xrdata1/mask_xrpic/a_232bf02f_7_l3.jpg
data/xrdata1/hand_xrpic/a_232bf02f_8_l4.jpg data/xrdata1/mask_xrpic/a_232bf02f_8_l4.jpg
data/xrdata1/hand_xrpic/a_232bf02f_9_l5.jpg data/xrdata1/mask_xrpic/a_232bf02f_9_l5.jpg
data/xrdata1/hand_xrpic/a_232bf02f_10_mc.jpg data/xrdata1/mask_xrpic/a_232bf02f_10_mc.jpg
data/xrdata1/hand_xrpic/a_232b

data/xrdata1/hand_xrpic/a_232bf02f_166_m2.jpg data/xrdata1/mask_xrpic/a_232bf02f_166_m2.jpg
data/xrdata1/hand_xrpic/a_232bf02f_167_m3.jpg data/xrdata1/mask_xrpic/a_232bf02f_167_m3.jpg
data/xrdata1/hand_xrpic/a_232bf02f_168_m4.jpg data/xrdata1/mask_xrpic/a_232bf02f_168_m4.jpg
data/xrdata1/hand_xrpic/a_232bf02f_169_m5.jpg data/xrdata1/mask_xrpic/a_232bf02f_169_m5.jpg
data/xrdata1/hand_xrpic/a_232bf02f_170_rc.jpg data/xrdata1/mask_xrpic/a_232bf02f_170_rc.jpg
data/xrdata1/hand_xrpic/a_232bf02f_171_ro.jpg data/xrdata1/mask_xrpic/a_232bf02f_171_ro.jpg
data/xrdata1/hand_xrpic/a_232bf02f_172_rq.jpg data/xrdata1/mask_xrpic/a_232bf02f_172_rq.jpg
data/xrdata1/hand_xrpic/a_232bf02f_173_rw.jpg data/xrdata1/mask_xrpic/a_232bf02f_173_rw.jpg
data/xrdata1/hand_xrpic/a_232bf02f_174_re.jpg data/xrdata1/mask_xrpic/a_232bf02f_174_re.jpg
data/xrdata1/hand_xrpic/a_232bf02f_175_r1.jpg data/xrdata1/mask_xrpic/a_232bf02f_175_r1.jpg
data/xrdata1/hand_xrpic/a_232bf02f_176_r2.jpg data/xrdata1/mask_xrpic/a_232bf02f

data/xrdata1/hand_xrpic/v_a_232bf02f_27_r3.jpg data/xrdata1/mask_xrpic/v_a_232bf02f_27_r3.jpg
data/xrdata1/hand_xrpic/v_a_232bf02f_28_r4.jpg data/xrdata1/mask_xrpic/v_a_232bf02f_28_r4.jpg
data/xrdata1/hand_xrpic/v_a_232bf02f_29_r5.jpg data/xrdata1/mask_xrpic/v_a_232bf02f_29_r5.jpg
data/xrdata1/hand_xrpic/v_a_232bf02f_30_lc.jpg data/xrdata1/mask_xrpic/v_a_232bf02f_30_lc.jpg
data/xrdata1/hand_xrpic/v_a_232bf02f_31_lo.jpg data/xrdata1/mask_xrpic/v_a_232bf02f_31_lo.jpg
data/xrdata1/hand_xrpic/v_a_232bf02f_32_lq.jpg data/xrdata1/mask_xrpic/v_a_232bf02f_32_lq.jpg
data/xrdata1/hand_xrpic/v_a_232bf02f_33_lw.jpg data/xrdata1/mask_xrpic/v_a_232bf02f_33_lw.jpg
data/xrdata1/hand_xrpic/v_a_232bf02f_34_le.jpg data/xrdata1/mask_xrpic/v_a_232bf02f_34_le.jpg
data/xrdata1/hand_xrpic/v_a_232bf02f_35_l1.jpg data/xrdata1/mask_xrpic/v_a_232bf02f_35_l1.jpg
data/xrdata1/hand_xrpic/v_a_232bf02f_36_l2.jpg data/xrdata1/mask_xrpic/v_a_232bf02f_36_l2.jpg
data/xrdata1/hand_xrpic/v_a_232bf02f_37_l3.jpg data/xrdata1/

data/xrdata1/hand_xrpic/v_a_232bf02f_186_l2.jpg data/xrdata1/mask_xrpic/v_a_232bf02f_186_l2.jpg
data/xrdata1/hand_xrpic/v_a_232bf02f_187_l3.jpg data/xrdata1/mask_xrpic/v_a_232bf02f_187_l3.jpg
data/xrdata1/hand_xrpic/v_a_232bf02f_188_l4.jpg data/xrdata1/mask_xrpic/v_a_232bf02f_188_l4.jpg
data/xrdata1/hand_xrpic/v_a_232bf02f_189_l5.jpg data/xrdata1/mask_xrpic/v_a_232bf02f_189_l5.jpg
data/xrdata1/hand_xrpic/v_a_232bf02f_190_mc.jpg data/xrdata1/mask_xrpic/v_a_232bf02f_190_mc.jpg
data/xrdata1/hand_xrpic/v_a_232bf02f_191_mo.jpg data/xrdata1/mask_xrpic/v_a_232bf02f_191_mo.jpg
data/xrdata1/hand_xrpic/v_a_232bf02f_192_mq.jpg data/xrdata1/mask_xrpic/v_a_232bf02f_192_mq.jpg
data/xrdata1/hand_xrpic/v_a_232bf02f_193_mw.jpg data/xrdata1/mask_xrpic/v_a_232bf02f_193_mw.jpg
data/xrdata1/hand_xrpic/v_a_232bf02f_194_me.jpg data/xrdata1/mask_xrpic/v_a_232bf02f_194_me.jpg
data/xrdata1/hand_xrpic/v_a_232bf02f_195_m1.jpg data/xrdata1/mask_xrpic/v_a_232bf02f_195_m1.jpg
data/xrdata1/hand_xrpic/v_a_232bf02f_196

data/xrdata2/hand_xrpic/a_4e7de01d_52_rq.jpg data/xrdata2/mask_xrpic/a_4e7de01d_52_rq.jpg
data/xrdata2/hand_xrpic/a_4e7de01d_53_rw.jpg data/xrdata2/mask_xrpic/a_4e7de01d_53_rw.jpg
data/xrdata2/hand_xrpic/a_4e7de01d_54_re.jpg data/xrdata2/mask_xrpic/a_4e7de01d_54_re.jpg
data/xrdata2/hand_xrpic/a_4e7de01d_55_r1.jpg data/xrdata2/mask_xrpic/a_4e7de01d_55_r1.jpg
data/xrdata2/hand_xrpic/a_4e7de01d_56_r2.jpg data/xrdata2/mask_xrpic/a_4e7de01d_56_r2.jpg
data/xrdata2/hand_xrpic/a_4e7de01d_57_r3.jpg data/xrdata2/mask_xrpic/a_4e7de01d_57_r3.jpg
data/xrdata2/hand_xrpic/a_4e7de01d_58_r4.jpg data/xrdata2/mask_xrpic/a_4e7de01d_58_r4.jpg
data/xrdata2/hand_xrpic/a_4e7de01d_59_r5.jpg data/xrdata2/mask_xrpic/a_4e7de01d_59_r5.jpg
data/xrdata2/hand_xrpic/a_4e7de01d_60_lc.jpg data/xrdata2/mask_xrpic/a_4e7de01d_60_lc.jpg
data/xrdata2/hand_xrpic/a_4e7de01d_61_lo.jpg data/xrdata2/mask_xrpic/a_4e7de01d_61_lo.jpg
data/xrdata2/hand_xrpic/a_4e7de01d_62_lq.jpg data/xrdata2/mask_xrpic/a_4e7de01d_62_lq.jpg
data/xrdat

...

data/xrdata4/hand_xrpic/a_c84316f5_581_mc.jpg data/xrdata4/mask_xrpic/a_c84316f5_581_mc.jpg
data/xrdata4/hand_xrpic/a_c84316f5_582_mo.jpg data/xrdata4/mask_xrpic/a_c84316f5_582_mo.jpg
data/xrdata4/hand_xrpic/a_c84316f5_583_mq.jpg data/xrdata4/mask_xrpic/a_c84316f5_583_mq.jpg
data/xrdata4/hand_xrpic/a_c84316f5_584_mw.jpg data/xrdata4/mask_xrpic/a_c84316f5_584_mw.jpg
data/xrdata4/hand_xrpic/a_c84316f5_585_me.jpg data/xrdata4/mask_xrpic/a_c84316f5_585_me.jpg
data/xrdata4/hand_xrpic/a_c84316f5_586_m1.jpg data/xrdata4/mask_xrpic/a_c84316f5_586_m1.jpg
data/xrdata4/hand_xrpic/a_c84316f5_587_m2.jpg data/xrdata4/mask_xrpic/a_c84316f5_587_m2.jpg
data/xrdata4/hand_xrpic/a_c84316f5_588_m3.jpg data/xrdata4/mask_xrpic/a_c84316f5_588_m3.jpg
data/xrdata4/hand_xrpic/a_c84316f5_589_m4.jpg data/xrdata4/mask_xrpic/a_c84316f5_589_m4.jpg
data/xrdata4/hand_xrpic/a_c84316f5_590_m5.jpg data/xrdata4/mask_xrpic/a_c84316f5_590_m5.jpg
data/xrdata4/hand_xrpic/a_c84316f5_591_rc.jpg data/xrdata4/mask_xrpic/a_c84316f5

data/xrdata4/hand_xrpic/v_a_c84316f5_89_r5.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_89_r5.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_90_lc.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_90_lc.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_91_lo.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_91_lo.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_92_lq.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_92_lq.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_93_lw.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_93_lw.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_94_le.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_94_le.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_95_l1.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_95_l1.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_96_l2.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_96_l2.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_97_l3.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_97_l3.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_98_l4.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_98_l4.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_99_l5.jpg data/xrdata4/

data/xrdata4/hand_xrpic/v_a_c84316f5_257_m3.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_257_m3.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_258_m4.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_258_m4.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_259_m5.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_259_m5.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_260_rc.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_260_rc.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_261_ro.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_261_ro.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_262_rq.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_262_rq.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_263_rw.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_263_rw.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_264_re.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_264_re.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_265_r1.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_265_r1.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_266_r2.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_266_r2.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_267

data/xrdata4/hand_xrpic/v_a_c84316f5_431_mc.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_431_mc.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_432_mo.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_432_mo.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_433_mq.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_433_mq.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_434_mw.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_434_mw.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_435_me.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_435_me.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_436_m1.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_436_m1.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_437_m2.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_437_m2.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_438_m3.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_438_m3.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_439_m4.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_439_m4.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_440_m5.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_440_m5.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_441

data/xrdata4/hand_xrpic/v_a_c84316f5_597_r2.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_597_r2.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_598_r3.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_598_r3.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_599_r4.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_599_r4.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_600_r5.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_600_r5.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_601_lc.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_601_lc.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_602_lo.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_602_lo.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_603_lq.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_603_lq.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_604_lw.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_604_lw.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_605_le.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_605_le.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_606_l1.jpg data/xrdata4/mask_xrpic/v_a_c84316f5_606_l1.jpg
data/xrdata4/hand_xrpic/v_a_c84316f5_607

In [29]:
pd.DataFrame(processed_hand).to_csv('complete_hand_data_clean', index=False)
pd.DataFrame(processed_mask).to_csv('complete_mask_data_clean', index=False)